# EDS Block Experimentation in JSLab

This notebook demonstrates how to load and run EDS blocks in a Jupyter environment using JSLab.

## Important Notes

1. **Run Cell 1 first** to initialize the DOM environment
2. **Pure EDS blocks only** - This notebook works best with pure EDS blocks that use vanilla JavaScript and standard DOM APIs
3. **Web Components not supported** - Blocks using Web Components (like custom elements) won't work in this jsdom environment
4. **Interactive features limited** - Button clicks and other interactions won't work in notebook output. Use `saveBlockHTML()` to save blocks as HTML files for full browser testing
5. **Visual output** - Use the `saveBlockHTML()` helper to generate styled HTML files saved to the `ipynb-tests/` folder that you can open in your browser

In [1]:
// ============================================================================
// SETUP: Initialize DOM environment and helper functions
// Run this cell FIRST before any other code!
// ============================================================================

// Load jsdom and set up DOM
const { JSDOM } = require('jsdom');

// Create virtual DOM
const dom = new JSDOM('<!DOCTYPE html><html><head></head><body></body></html>', {
  url: 'http://localhost',
  pretendToBeVisual: true
});

// Make DOM globals available
global.document = dom.window.document;
global.window = dom.window;
global.HTMLElement = dom.window.HTMLElement;
global.Element = dom.window.Element;
global.Node = dom.window.Node;
global.customElements = dom.window.customElements;
global.CustomEvent = dom.window.CustomEvent;
global.Event = dom.window.Event;

console.log('✓ DOM environment initialized');
console.log('✓ Web Components (customElements) enabled');

// Ensure output directory exists
const fs = require('fs');
const outputDir = './ipynb-tests';
if (!fs.existsSync(outputDir)) {
  fs.mkdirSync(outputDir, { recursive: true });
  console.log(`✓ Created output directory: ${outputDir}`);
} else {
  console.log(`✓ Output directory ready: ${outputDir}`);
}

console.log('✓ Will link CSS files from /styles and /blocks folders');

// ----------------------------------------------------------------------------
// Helper: Load CSS for a block
// ----------------------------------------------------------------------------
async function loadBlockStyles(blockName) {
  const fs = await import('fs/promises');
  const path = await import('path');
  
  const cssPath = path.resolve(`./blocks/${blockName}/${blockName}.css`);
  try {
    const css = await fs.readFile(cssPath, 'utf-8');
    const style = global.document.createElement('style');
    style.textContent = css;
    global.document.head.appendChild(style);
    console.log(`✓ Loaded styles for ${blockName}`);
    return css;
  } catch (e) {
    console.log(`ℹ No CSS file found for ${blockName}`);
    return null;
  }
}

global.loadBlockStyles = loadBlockStyles;
console.log('✓ loadBlockStyles() helper ready');

// ----------------------------------------------------------------------------
// Helper: Test any block with content
// ----------------------------------------------------------------------------
async function testBlock(blockName, innerHTML = '') {
  console.log(`\n=== Testing: ${blockName} ===`);
  
  try {
    // Load the block module
    const path = await import('path');
    const modulePath = path.resolve(`./blocks/${blockName}/${blockName}.js`);
    const module = await import(modulePath);
    const decorate = module.default;
    
    // Load styles (for internal testing)
    const css = await loadBlockStyles(blockName);
    
    // Create block element
    const block = global.document.createElement('div');
    block.className = blockName;
    
    // Add content if provided
    if (innerHTML) {
      block.innerHTML = innerHTML;
    }
    
    console.log('Before:', block.innerHTML || '(empty)');
    
    // Decorate the block
    await decorate(block);
    
    const after = block.innerHTML || '';
    console.log('After:', after.substring(0, 100) + (after.length > 100 ? '...' : ''));
    
    return block;
  } catch (error) {
    console.error(`✗ Error testing ${blockName}:`, error.message);
    console.error(`   Block: ${blockName} may require browser-specific features`);
    throw error;
  }
}

global.testBlock = testBlock;
console.log('✓ testBlock() helper ready');

// ----------------------------------------------------------------------------
// Helper: Check if block CSS file exists
// ----------------------------------------------------------------------------
async function hasBlockCSS(blockName) {
  const fs = await import('fs/promises');
  const path = await import('path');
  const cssPath = path.resolve(`./blocks/${blockName}/${blockName}.css`);
  try {
    await fs.access(cssPath);
    return true;
  } catch {
    return false;
  }
}

// ----------------------------------------------------------------------------
// Helper: Save block as HTML file for browser viewing
// ----------------------------------------------------------------------------
async function saveBlockHTML(blockName, innerHTML = '', filename = null) {
  const block = await testBlock(blockName, innerHTML);
  
  // Check if block has CSS file
  const blockHasCSS = await hasBlockCSS(blockName);
  
  // Create block CSS link if file exists
  const blockCSSLink = blockHasCSS 
    ? `<link rel="stylesheet" href="../blocks/${blockName}/${blockName}.css">`
    : '<!-- No block CSS file -->';
  
  // Create a complete HTML document that links to CSS files
  const html = `<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>${blockName} Block Test</title>
  
  <!-- Global EDS Styles -->
  <link rel="stylesheet" href="../styles/styles.css">
  <link rel="stylesheet" href="../styles/fonts.css">
  <link rel="stylesheet" href="../styles/lazy-styles.css">
  
  <!-- Block-specific styles -->
  ${blockCSSLink}
  
  <!-- Preview container styles -->
  <style>
    body {
      display: block !important;
      padding: 20px;
      background: #f5f5f5;
    }
    
    .preview-container {
      max-width: 1200px;
      margin: 0 auto;
      background: white;
      padding: 40px;
      border-radius: 8px;
      box-shadow: 0 2px 8px rgba(0,0,0,0.1);
    }
    
    .preview-header {
      border-bottom: 2px solid var(--light-color, #eee);
      padding-bottom: 20px;
      margin-bottom: 40px;
    }
    
    .preview-header h1 {
      margin: 0;
      color: var(--text-color, black);
    }
  </style>
</head>
<body class="appear">
  <div class="preview-container">
    <div class="preview-header">
      <h1>${blockName} Block Preview</h1>
    </div>
    <main>
      <div class="section">
        <div>
          ${block.outerHTML}
        </div>
      </div>
    </main>
  </div>
</body>
</html>`;
  
  // Save to ipynb-tests directory
  const fs = await import('fs/promises');
  const path = await import('path');
  const outputFile = filename || `${blockName}-preview.html`;
  const outputPath = path.resolve('./ipynb-tests', outputFile);
  
  await fs.writeFile(outputPath, html, 'utf-8');
  console.log(`\n✓ Saved HTML preview to: ipynb-tests/${outputFile}`);
  console.log(`  Open this file in your browser to see the styled result!`);
  
  return outputPath;
}

global.saveBlockHTML = saveBlockHTML;
console.log('✓ saveBlockHTML() helper ready');

console.log('\n========================================');
console.log('Setup complete! Ready to test EDS blocks');
console.log('========================================\n');

'Setup complete! Run the cells below to test blocks.'

✓ DOM environment initialized
✓ Web Components (customElements) enabled
✓ Created output directory: ./ipynb-tests
✓ Will link CSS files from /styles and /blocks folders
✓ loadBlockStyles() helper ready
✓ testBlock() helper ready
✓ saveBlockHTML() helper ready

Setup complete! Ready to test EDS blocks

Setup complete! Run the cells below to test blocks.


## Part 1: Simple Tests

Test basic DOM functionality and simple blocks.

In [2]:
// Simple test - create a DOM element and display it
const testDiv = global.document.createElement('div');
testDiv.textContent = 'Hello from DOM!';
testDiv.className = 'test';

console.log('✓ Created element:', testDiv.outerHTML);

// Return the HTML to display in JSLab
testDiv.outerHTML

✓ Created element: <div class="test">Hello from DOM!</div>
<div class="test">Hello from DOM!</div>


## Part 2: Testing Blocks with Content

Test blocks that require specific content structures.

In [4]:
// Example 1: Test HelloWorld block (no content needed)
const helloBlock = await global.testBlock('helloworld');
helloBlock.outerHTML


=== Testing: helloworld ===


✓ Loaded styles for helloworld
Before: (empty)
After: <div>Hello World</div>
<div class="helloworld"><div>Hello World</div></div>


In [ ]:
// Example 2: Test Accordion block with content
// Structure: Each row needs 2 divs (label and body)
const accordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
`;

const accordionBlock = await global.testBlock('accordion', accordionContent);
console.log('Accordion sections created:', accordionBlock.querySelectorAll('details').length);
accordionBlock.outerHTML

## Part 2b: Visual Output

Save blocks as HTML files to the `ipynb-tests/` folder to view styled results in your browser.

In [5]:
// Example 3: Save Accordion block as HTML file
const styledAccordionContent = `
  <div>
    <div>What is EDS?</div>
    <div>Edge Delivery Services is Adobe's modern web platform for building fast, performant websites.</div>
  </div>
  <div>
    <div>How do blocks work?</div>
    <div>Blocks are JavaScript functions that decorate DOM elements and transform content structure.</div>
  </div>
  <div>
    <div>Why use JSLab?</div>
    <div>JSLab lets you experiment with blocks in a notebook environment for rapid testing and development.</div>
  </div>
`;

// Saves to ipynb-tests/accordion-preview.html
await global.saveBlockHTML('accordion', styledAccordionContent);


=== Testing: accordion ===


✗ Error testing accordion: Cannot find module '/Users/tomcranstoun/Documents/GitHub/webcomponents-with-eds/blocks/accordion/accordion.js'
Require stack:
- /Users/tomcranstoun/Documents/GitHub/webcomponents-with-eds/src.js
   Block: accordion may require browser-specific features
Error: Cannot find module '/Users/tomcranstoun/Documents/GitHub/webcomponents-with-eds/blocks/accordion/accordion.js'
Require stack:
- /Users/tomcranstoun/Documents/GitHub/webcomponents-with-eds/src.js
    at Function._resolveFilename (node:internal/modules/cjs/loader:1383:15)
    at defaultResolveImpl (node:internal/modules/cjs/loader:1025:19)
    at resolveForCJSWithHooks (node:internal/modules/cjs/loader:1030:22)
    at Function._load (node:internal/modules/cjs/loader:1192:37)
    at TracingChannel.traceSync (node:diagnostics_channel:328:14)
    at wrapModuleLoad (node:internal/modules/cjs/loader:237:24)
    at Module.require (node:internal/modules/cjs/loader:1463:12)
    at require (node:internal/modules/he

## Part 3: Discover Available Blocks

List all blocks available for testing in this project.

In [6]:
// List all available blocks
const fs = require('fs').promises;
const blocksDir = './blocks';

const entries = await fs.readdir(blocksDir, { withFileTypes: true });
const blocks = entries
  .filter(entry => entry.isDirectory())
  .map(entry => entry.name)
  .sort();

console.log(`Found ${blocks.length} blocks:\n`);
console.log(blocks.join('\n'));

blocks

Found 7 blocks:

accordion
cards
columns
counter
helloworld
shoelace
shoelace-card
[
  'accordion',
  'cards',
  'columns',
  'counter',
  'helloworld',
  'shoelace',
  'shoelace-card'
]


## Quick Reference

### Usage

**Test blocks (returns DOM element):**
```javascript
const block = await testBlock('blockname', '<div>content</div>');
block.outerHTML
```

**Save styled preview to HTML file:**
```javascript
// Saves to ipynb-tests/blockname-preview.html
await saveBlockHTML('blockname', '<div>content</div>');

// Custom filename (still saved to ipynb-tests/)
await saveBlockHTML('blockname', '<div>content</div>', 'my-test.html');
```

### Tips
1. Always run **Cell 1** first to initialize the DOM environment
2. Use `testBlock()` to test blocks and see raw HTML output in the notebook
3. Use `saveBlockHTML()` to save styled previews to `ipynb-tests/` folder
4. Open saved HTML files in your browser to see fully styled, interactive blocks
5. Check console logs to see before/after transformation
6. All HTML output files are saved to the `ipynb-tests/` directory

### Available Helper Functions
- `testBlock(blockName, innerHTML)` - Test a block and return DOM element
- `saveBlockHTML(blockName, innerHTML, filename?)` - Save styled HTML to `ipynb-tests/` folder
- `loadBlockStyles(blockName)` - Load CSS styles for a block